# Bokeh and widgets

* [The widgets can be seen at mybinder.org by clicking here](https://mybinder.org/v2/gh/tlinnet/mybinder_relax/master?filepath=bokeh_dashboard.ipynb)
* [Reproduced from here](https://github.com/bokeh/bokeh/blob/0.12.10/examples/howto/layouts/dashboard.py)
* [And here](https://github.com/bokeh/bokeh/blob/master/examples/howto/notebook_comms/Jupyter%20Interactors.ipynb)

In [1]:
import numpy as np

import bokeh.plotting as bplt
from bokeh.layouts import layout
from bokeh.models import CustomJS, Slider, ColumnDataSource, WidgetBox
from bokeh.io import output_notebook, push_notebook
output_notebook()

# Widgets
import ipywidgets as w

Loading BokehJS ...

In [2]:
def bollinger():
    # Define Bollinger Bands.
    upperband = np.random.randint(100, 150, size=100)
    lowerband = upperband - 100
    x_data = np.arange(1, 101)

    # Bollinger shading glyph:
    band_x = np.append(x_data, x_data[::-1])
    band_y = np.append(lowerband, upperband[::-1])

    p = bplt.figure(plot_width=600, plot_height=200, x_axis_type='datetime', tools='pan')
    p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)

    p.title.text = 'Bollinger Bands'
    p.title_location = 'left'
    p.title.align = 'left'
    #p.plot_height = 600
    #p.plot_width = 800
    p.grid.grid_line_alpha = 0.4
    return [p]


def slider():
    x = np.linspace(0, 10, 100)
    y = np.sin(x)

    #source = ColumnDataSource(data=dict(x=x, y=y))
    source = ColumnDataSource(data={'x':x, 'y':y})
    
    plot = bplt.figure(plot_width=300, plot_height=200,
        y_range=(-10, 10), tools='', toolbar_location=None,
        title="Sliders example")
    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

    callback = CustomJS(args=dict(source=source), code="""
        var data = source.data;
        var A = amp.value;
        var k = freq.value;
        var phi = phase.value;
        var B = offset.value;
        x = data['x']
        y = data['y']
        for (i = 0; i < x.length; i++) {
            y[i] = B + A*Math.sin(k*x[i]+phi);
        }
        source.change.emit();
    """)

    #amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude", callback=callback, callback_policy='mouseup')
    amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude", callback=callback)
    callback.args["amp"] = amp_slider

    freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency", callback=callback)
    callback.args["freq"] = freq_slider

    phase_slider = Slider(start=0, end=6.4, value=0, step=.1, title="Phase", callback=callback)
    callback.args["phase"] = phase_slider

    offset_slider = Slider(start=-5, end=5, value=0, step=.1, title="Offset", callback=callback)
    callback.args["offset"] = offset_slider

    widgets = WidgetBox(amp_slider, freq_slider, phase_slider, offset_slider)
    return [widgets, plot]


def linked_panning():
    N = 100
    x = np.linspace(0, 4 * np.pi, N)
    y1 = np.sin(x)
    y2 = np.cos(x)
    y3 = np.sin(x) + np.cos(x)

    s1 = bplt.figure(plot_width=200, plot_height=200, tools='pan')
    s1.circle(x, y1, color="navy", size=8, alpha=0.5)
    s2 = bplt.figure(plot_width=200, plot_height=200, tools='pan', x_range=s1.x_range, y_range=s1.y_range)
    s2.circle(x, y2, color="firebrick", size=8, alpha=0.5)
    s3 = bplt.figure(plot_width=200, plot_height=200, tools='pan, box_select', x_range=s1.x_range)
    s3.circle(x, y3, color="olive", size=8, alpha=0.5)
    return [s1, s2, s3]



In [3]:
l = layout([
    bollinger(),
    slider(),
    linked_panning(),
#], sizing_mode='stretch_both')
])

In [4]:
bplt.show(l)

In [5]:
def sin_calc(xt=None, A=None, l=None, o=None, p=None):
    """
    @keyword xt:        The array of time points.
    @keyword A:         The initial amplitude of the envelope.
    @keyword l:         l=lambda, the decay constant, in the reciprocal of the time units of the X axis.
    @keyword o:         o=omega, the angular frequency.
    @keyword p:         p=phi, the phase angle at some arbitrary point.
    """
    # Calculate y
    y = A*np.exp(-l*xt)*np.cos(o*xt+p)
    return y

def sin_plot(figp=None, Amp_1=1., lambda_1=1., omega_1=2*np.pi, phi_1=0.0):
    """
    @keyword Amp:       The initial amplitude of the envelope.
    @keyword lambd:     lambda, the decay constant, in the reciprocal of the time units of the X axis.
    @keyword omega:     omega, the angular frequency.
    @keyword phi:       phi, the phase angle at some arbitrary point.
    """
    # Make array of time points
    xt = np.linspace(0, 5, num=1000)

    # Gety values
    y_1 = sin_calc(xt=xt, A=Amp_1, l=lambda_1, o=omega_1, p=phi_1)

    # Assign data
    CDS = ColumnDataSource(data={'x':xt, 'y':y_1})
    if figp:
        # Update the figure plot
        figp.data_source.data['y'] = y_1
        push_notebook()
    else:
        return CDS

def make_widget_2():
    # Create for first graph
    A1 = w.FloatSlider(value=1.0, description='A 1', max=5.0, min=0.1)
    l1 = w.FloatSlider(value=1.0, description=r'\(\lambda\) 1', max=5.0)
    o1 = w.FloatSlider(value=2*np.pi, description=r'\(\omega\) 1', max=4*np.pi)
    p1 = w.FloatSlider(value=0.0, description=r'\(\phi\) 1', max=round(2*np.pi, 1), min=-round(2*np.pi, 1))
    # Put sliders under each other
    sliders_1 = w.VBox([A1, l1, o1, p1])
    # Put next to each other
    ui = w.HBox([sliders_1])
    
    # Make initial figure
    fig = bplt.figure(plot_width=400, plot_height=400, title="Sliders example")
    source = sin_plot()
    # Plot first data
    figp = fig.line('x', 'y', source=source, line_width=3, line_alpha=0.6)
    bplt.show(fig, notebook_handle=True)

    # Match input variables with sliders
    d = {'figp': w.fixed(figp),
        'Amp_1': A1, 'lambda_1': l1,  'omega_1': o1, 'phi_1': p1}

    # Get the output figure
    out = w.interactive_output(sin_plot, d)

    return ui, out

In [6]:
widget_2 = make_widget_2()
display(*widget_2)

HBox(children=(VBox(children=(FloatSlider(value=1.0, description='A 1', max=5.0, min=0.1), FloatSlider(value=1.0, description='\\(\\lambda\\) 1', max=5.0), FloatSlider(value=6.283185307179586, description='\\(\\omega\\) 1', max=12.566370614359172), FloatSlider(value=0.0, description='\\(\\phi\\) 1', max=6.3, min=-6.3))),))

Output()